App Profitability Project

The goal is this project is analyze a set of app data to determine what apps would be most profitable for a pretend group of app designers. All apps that our pretend company makes are free to download and play, so all of the company's revenue comes from in-app ads, so we can to figure out which types app wil bring in and retain the most potential users for our company.

In [1]:
from csv import reader
#Google Data
opened_file = open("googleplaystore.csv")
read_file = reader(opened_file)
android = list(read_file)
android_header = android[0]
android = android[1:]

#iOS Data
opened_file = open("AppleStore.csv")
read_file = reader(opened_file)
ios = list(read_file)
ios_header = ios[0]
ios_data = ios[1:]

In [2]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

#Google
print(android_header)#category, rating, reviews, installs, type and genres will be useful columns
print("\n")
explore_data(android, 1, 5, True)
del android[10472]
#iOS
print(ios_header) # rating counts and genre will be useful columns
print("\n")
explore_data(ios_data, 1, 5, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


Number of rows: 10841
Number of columns: 13
['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count

As we sort through this data, one thing we find is that in the Google Play dataset there are duplicate apps littered throughout. We can see an example of this below

In [3]:
duplicate_apps = []
unique_apps = []

for app in android:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
print ("Number of duplicate apps:", len(duplicate_apps))
print ("Examples of duplicate apps:", duplicate_apps[:10])

Number of duplicate apps: 1181
Examples of duplicate apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack']


Now that we know the number of, and the names of some of these duplicate apps, let's examine the specifics of one of them

In [4]:
for app in android:
    name = app[0]
    if name == "Quick PDF Scanner + OCR FREE":
        print (app)

['Quick PDF Scanner + OCR FREE', 'BUSINESS', '4.2', '80805', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'February 26, 2018', 'Varies with device', '4.0.3 and up']
['Quick PDF Scanner + OCR FREE', 'BUSINESS', '4.2', '80805', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'February 26, 2018', 'Varies with device', '4.0.3 and up']
['Quick PDF Scanner + OCR FREE', 'BUSINESS', '4.2', '80804', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'February 26, 2018', 'Varies with device', '4.0.3 and up']


We can see that in this instance that the main factor causing the duplicate entries is the number of ratings (80804 vs 80805). Using this information, we will make the number of ratings our criterion for removing duplicate data and keep only the entry with the highest number of reviews, since we can assume that is the most recent entry.

In [5]:
reviews_max = {}

for app in android:
    name = app[0]
    n_reviews = float(app[3])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews

In [6]:
android_clean = []
already_added = []

for app in android:
    name = app[0]
    n_reviews = float(app[3])
    if (reviews_max[name] == n_reviews) and (name not in already_added):
        android_clean.append(app)
        already_added.append(name)    

This step allowed us to remove all duplicate apps from our data based on the criteria discussed above

In [7]:
def char_check(a_string):
    non_ascii = 0
    for char in a_string:
        if ord(char) > 127:
            non_ascii += 1
        if non_ascii > 3:
            return False
    return True

In [8]:
android_english = []
ios_english = []

for app in android_clean:
    name = app[0]
    if char_check(name):
        android_english.append(app)
for app in ios:
    name = app[0]
    if char_check(name):
        ios_english.append(app)    

Since our company is creating apps for a primaryily English speaking audience, we wanted to filter out any apps that were seemingly not English apps. We created a function to identify if 3 or more characters were outside of the list of commonly used English charactersand, if so, remove them from our updated app list.

In [9]:
android_free = []
ios_free = []
android_not_free = []
ios_not_free = []

for app in android_english:
    price = app[7]
    if price == "0":
        android_free.append(app)
    else:
        android_not_free.append(app)
        
for app in ios_english:
    price = app[4]
    if price == "0.0":
        ios_free.append(app)
    else:
        ios_not_free.append(app)
        
print(len(android_free))
print(len(android_not_free))
print("\n")
print(len(ios_free))
print(len(ios_not_free))

8864
750


4056
3142


With all of that, we've cleaned our data, including:
1. Removing any inaccurate date
2. Removing any duplicate app entries
3. Removing any non-English apps
4. Removing any non-free apps

As we mentioned at the beginning, the goal our project is to find out what sort of app would attract the most users and, in turn, gennerate the most profit for our company through the use of in-app ads. For this, we will focus on a 3-point strategy
1. Figure out which genre of app is most popular on Google Play and build a minimal version app
2. If the app generate enough user interest, we will update the app further
3. If the app is profitable after 6 months, we will develop an iOS version of the app

After inspecting both data sets, we can determine the best way to go is to inspect the "Genres" and "Category" column for Google Play and "prime_genres" for iOS to determine the most common genre in each market



In [10]:
def freq_table(dataset, index):
    f_table = {}
    total = 0
    for app in dataset:
        genre = app[index]
        total += 1
        if genre in f_table:
            f_table[genre] += 1
        else:
            f_table[genre] = 1
            
    table_percentages = {}
    for key in f_table:
        percentage = (f_table[key] / total) * 100
        table_percentages[key] = round(percentage, 3)
    return table_percentages
        
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [11]:
#Google Play "Genre" column
display_table(android_free, -4)

Tools : 8.45
Entertainment : 6.069
Education : 5.347
Business : 4.592
Productivity : 3.892
Lifestyle : 3.892
Finance : 3.7
Medical : 3.531
Sports : 3.463
Personalization : 3.317
Communication : 3.238
Action : 3.102
Health & Fitness : 3.08
Photography : 2.944
News & Magazines : 2.798
Social : 2.662
Travel & Local : 2.324
Shopping : 2.245
Books & Reference : 2.144
Simulation : 2.042
Dating : 1.861
Arcade : 1.85
Video Players & Editors : 1.771
Casual : 1.76
Maps & Navigation : 1.399
Food & Drink : 1.241
Puzzle : 1.128
Racing : 0.993
Role Playing : 0.936
Libraries & Demo : 0.936
Auto & Vehicles : 0.925
Strategy : 0.914
House & Home : 0.824
Weather : 0.801
Events : 0.711
Adventure : 0.677
Comics : 0.609
Beauty : 0.598
Art & Design : 0.598
Parenting : 0.496
Card : 0.451
Casino : 0.429
Trivia : 0.417
Educational;Education : 0.395
Board : 0.384
Educational : 0.372
Education;Education : 0.338
Word : 0.259
Casual;Pretend Play : 0.237
Music : 0.203
Racing;Action & Adventure : 0.169
Puzzle;Brain G

In [12]:
#Google Play "Categories" column
display_table(android_free, 1)

FAMILY : 18.908
GAME : 9.725
TOOLS : 8.461
BUSINESS : 4.592
LIFESTYLE : 3.903
PRODUCTIVITY : 3.892
FINANCE : 3.7
MEDICAL : 3.531
SPORTS : 3.396
PERSONALIZATION : 3.317
COMMUNICATION : 3.238
HEALTH_AND_FITNESS : 3.08
PHOTOGRAPHY : 2.944
NEWS_AND_MAGAZINES : 2.798
SOCIAL : 2.662
TRAVEL_AND_LOCAL : 2.335
SHOPPING : 2.245
BOOKS_AND_REFERENCE : 2.144
DATING : 1.861
VIDEO_PLAYERS : 1.794
MAPS_AND_NAVIGATION : 1.399
FOOD_AND_DRINK : 1.241
EDUCATION : 1.162
ENTERTAINMENT : 0.959
LIBRARIES_AND_DEMO : 0.936
AUTO_AND_VEHICLES : 0.925
HOUSE_AND_HOME : 0.824
WEATHER : 0.801
EVENTS : 0.711
PARENTING : 0.654
ART_AND_DESIGN : 0.643
COMICS : 0.62
BEAUTY : 0.598


In [13]:
#iOS "prime_genres" column
display_table(ios_free, -5)

Games : 55.646
Entertainment : 8.235
Photo & Video : 4.117
Social Networking : 3.526
Education : 3.254
Shopping : 2.983
Utilities : 2.687
Lifestyle : 2.318
Finance : 2.071
Sports : 1.948
Health & Fitness : 1.874
Music : 1.652
Book : 1.627
Productivity : 1.529
News : 1.43
Travel : 1.381
Food & Drink : 1.06
Weather : 0.764
Reference : 0.493
Navigation : 0.493
Business : 0.493
Catalogs : 0.222
Medical : 0.197


Analyzing the two Google Play datasets, we can see that the two genres that are the most popular across both are "Tools" and "Game" or "Entertainment". One other interesting thing we see is that, while both datasets are simmilar, the "Genres" column seems to provide a much more granual breakdown of apps, including several sections for educational apps alone. Since our company is mainly looking at a big picture persepctive for the genre that will make the most profit, we'll use information from the "Categories" column. In this column, we see that most of the apps are under the "Family" category which, if we look closer, we see are mostly games targeted at children. This lines up with the iOS app store's most popular category being games (55%) as well, however just because that is the category with the highest number of apps does not mean that it has the most installs.

Since we know that the number of apps does not directly correlate to the number of installs, we want to analyze the total number of installs for iOS and Google Play. We've done so below by isolating the number of user ratings and averaging them as a workaround.

In [14]:
ios_genres = freq_table(ios_free, -5)

for genre in ios_genres:
    total = 0
    len_genre = 0
    for app in ios_free:
        genre_app = app[-5]
        if genre_app == genre:            
            n_ratings = float(app[5])
            total += n_ratings
            len_genre += 1
    avg_n_ratings = total / len_genre
    print(genre, ':', avg_n_ratings)

Social Networking : 53078.195804195806
Photo & Video : 27249.892215568863
Games : 18924.68896765618
Music : 56482.02985074627
Reference : 67447.9
Health & Fitness : 19952.315789473683
Weather : 47220.93548387097
Utilities : 14010.100917431193
Travel : 20216.01785714286
Shopping : 18746.677685950413
News : 15892.724137931034
Navigation : 25972.05
Lifestyle : 8978.308510638299
Entertainment : 10822.961077844311
Food & Drink : 20179.093023255813
Sports : 20128.974683544304
Book : 8498.333333333334
Finance : 13522.261904761905
Education : 6266.333333333333
Productivity : 19053.887096774193
Business : 6367.8
Catalogs : 1779.5555555555557
Medical : 459.75


We'll now do the same thing for Google Play data

In [18]:
categories_android = freq_table(android_free, 1)

for category in categories_android:
    total = 0
    len_category = 0
    for app in android_free:
        category_app = app[1]
        if category_app == category:            
            n_installs = app[5]
            n_installs = n_installs.replace(',', '')
            n_installs = n_installs.replace('+', '')
            total += float(n_installs)
            len_category += 1
    avg_n_installs = total / len_category
    print(category, ':', avg_n_installs)

ART_AND_DESIGN : 1986335.0877192982
AUTO_AND_VEHICLES : 647317.8170731707
BEAUTY : 513151.88679245283
BOOKS_AND_REFERENCE : 8767811.894736841
BUSINESS : 1712290.1474201474
COMICS : 817657.2727272727
COMMUNICATION : 38456119.167247385
DATING : 854028.8303030303
EDUCATION : 1833495.145631068
ENTERTAINMENT : 11640705.88235294
EVENTS : 253542.22222222222
FINANCE : 1387692.475609756
FOOD_AND_DRINK : 1924897.7363636363
HEALTH_AND_FITNESS : 4188821.9853479853
HOUSE_AND_HOME : 1331540.5616438356
LIBRARIES_AND_DEMO : 638503.734939759
LIFESTYLE : 1437816.2687861272
GAME : 15588015.603248259
FAMILY : 3695641.8198090694
MEDICAL : 120550.61980830671
SOCIAL : 23253652.127118643
SHOPPING : 7036877.311557789
PHOTOGRAPHY : 17840110.40229885
SPORTS : 3638640.1428571427
TRAVEL_AND_LOCAL : 13984077.710144928
TOOLS : 10801391.298666667
PERSONALIZATION : 5201482.6122448975
PRODUCTIVITY : 16787331.344927534
PARENTING : 542603.6206896552
WEATHER : 5074486.197183099
VIDEO_PLAYERS : 24727872.452830188
NEWS_AND_